# Daily Challenge: Web Scraping and Data Visualization
## Your Task 

### Instructions

*Use this webpage : Tel Aviv - BBC Weather*
- Scrape the Webpage: Use Python libraries like Selenium to interact with the webpage and BeautifulSoup to parse the HTML, Extract relevant data such as dates, numerical values, categories, etc., Organize the scraped data into a structured format.

*Data Analysis with Pandas:*
- Load your scraped data into a Pandas DataFrame.
- Clean and preprocess the data if necessary (handling missing values, type conversion, etc.).
- Perform basic analysis, such as calculating averages, totals, or trends over time.

*Data Visualization with Seaborn and Matplotlib:*
- Use Seaborn and Matplotlib to create visualizations that provide insights into the data.
- Examples of visualizations can include line plots for trends over time, bar charts for comparing categories, or heatmaps for showing data density.
- Ensure your visualizations are well-labeled with clear titles, axis labels, and legends where appropriate.

*Document Your Findings:*
- Create a report summarizing your methodology, analysis, and insights from the visualizations.
- Explain any interesting patterns or insights derived from your data.

In [65]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-features=MediaCapabilities")
options.add_argument("--lang=en")
driver = webdriver.Chrome(options=options)


try:
    url = "https://www.bbc.com/weather/293397"
    driver.get(url)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "wr-day"))
    )
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    days = soup.find_all("li", class_="wr-day")

    weather_data = []

    for day in days:
        day_name = day.find("span", class_="wr-date__longish")
        day_name = day_name.text.strip() if day_name else "Tonight"
        day_temp = day.find('div', class_="wr-day__temperature")
        temp_high_div = day_temp.find("div", class_="wr-day-temperature__high") if day_temp else "N/A"

        temp_high = temp_high_div.find("span", class_="wr-value--temperature--c").text.strip() if temp_high_div else "N/A"
        temp_low_div = day.find("div", class_="wr-day-temperature__low")

        temp_low = temp_low_div.find("span", class_="wr-value--temperature--c").text.strip() if temp_high_div else "N/A"

        condition = day.find("div", class_="wr-weather-type__text").text.strip() 

        weather_data.append({
            "Day": day_name,
            "High Temperature (°C)": temp_high,
            "Low Temperature (°C)": temp_low,
            "Condition": condition
        })

    df = pd.DataFrame(weather_data)

    df.to_csv("bbc_weather_tel_aviv.csv", index=False)

    print(df)

finally:
    driver.quit()

         Day High Temperature (°C) Low Temperature (°C)           Condition
0    Tonight                   20°                   9°               Sunny
1   Wed 18th                   22°                   9°               Sunny
2   Thu 19th                   21°                  10°               Sunny
3   Fri 20th                   18°                  12°  Light Rain Showers
4   Sat 21st                   20°                  11°               Sunny
5   Sun 22nd                   19°                  11°               Sunny
6   Mon 23rd                   18°                  11°    Thundery Showers
7   Tue 24th                   18°                  10°     Sunny Intervals
8   Wed 25th                   19°                  11°  Light Rain Showers
9   Thu 26th                   17°                  11°  Heavy Rain Showers
10  Fri 27th                   17°                  10°     Sunny Intervals
11  Sat 28th                   17°                  10°  Light Rain Showers
12  Sun 29th

In [66]:
import plotly.express as px
import numpy as np
import pandas as pd
df = pd.read_csv("bbc_weather_tel_aviv.csv")
df_clean = df.dropna(subset=["High Temperature (°C)", "Low Temperature (°C)", "Day", "Condition"])

# Create a 3D scatter plot
fig = px.scatter_3d(df_clean, 
                    x="Day", 
                    y="Low Temperature (°C)", 
                    z="High Temperature (°C)", 
                    color="Condition", 
                    labels={"High Temperature (°C)": "High Temp (°C)", 
                            "Low Temperature (°C)": "Low Temp (°C)", 
                            "Day": "Day", 
                            "Condition": "Condition"})

# Set title and show the plot
fig.update_layout(title="3D Weather Data Scatter Plot", 
                  scene=dict(
                      xaxis_title="Day",
                      yaxis_title="Low Temp (°C)",
                      zaxis_title="High Temp (°C)"))

fig.show()

# Weather Data Analysis
- This dataset provides daily weather information for a period from "Tonight" through the 29th of the month, with high and low temperatures in Celsius and corresponding weather conditions.

*Temperature Trends:*

- The high temperatures range from 17°C to 22°C, with a noticeable decrease towards the end of the month, especially after the 26th, when the temperature consistently hovers around 17°C.
- The low temperatures fluctuate between 9°C and 12°C, with the cooler nights occurring on the 17th, 18th, and 29th, where the low temperatures drop to 9°C.
Weather Conditions:

- The majority of the days are categorized as either "Sunny" or "Sunny Intervals," indicating generally clear weather with occasional partial cloud cover.
- Rainy conditions are present in the middle of the period, with "Light Rain Showers" occurring on multiple days (20th, 23rd, 24th, 26th, 27th, 28th) and a "Heavy Rain Shower" on the 27th.
- The data suggests that while the beginning of the period was mostly sunny, rain became more frequent towards the end of the month.

- Summary: Overall, the dataset shows a combination of sunny and mild temperatures, with periods of rain beginning around the 20th and intensifying towards the end of the month. The general trend shows a slight cooling of temperatures as the month progresses, with the final days being marked by cooler temperatures and increasing rainfall.